In [3]:
%pip install python-dotenv
%pip install openai

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import openai
import os
from dotenv import load_dotenv

In [30]:
load_dotenv()
openai.api_key=os.getenv("OPENAI_API_KEY")
#client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", APIKEY))
client = openai.OpenAI(api_key=openai.api_key)

# Upload files for 70-30 job

In [31]:
#Upload dataset
#with open("70-30/train_data.jsonl", "rb") as training_fd:
training_response = client.files.create(
    file=open("70-30/train_data.jsonl", "rb"), purpose="fine-tune"
)

training_file_id = training_response.id

'''with open("70-30/validation_data.jsonl", "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id'''

print("Training file ID:", training_file_id)
#print("Validation file ID:", validation_file_id)



APIConnectionError: Connection error.

In [17]:
#Upload dataset
''''
train=openai.File.create(
    file=open("70-30/train_data.jsonl","rb"),
    purpose="fine-tune"
)

train'''

APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
